In [5]:
#Multivariate Analysis
#Import libraries
library(dplyr)
library(ggplot2)
library(rms)
library(mgcv)
library(tidymv)
library(tidyr)
library("ggthemes")
library(broom)
library(stringr)
library(R.utils)
library(data.table)

In [6]:
#Restricted analysis (only)

#Read the complete set (no imputation)
UKBB_AG2_m <- fread("~/jupyter/UKBB_AG2_12Jan21.txt", header = TRUE, na.strings=c("",".","NA")) %>% select(f.eid,T2D_status,ALBUMINERIA.0.0,
                                 ESKD.0.0,CKD.0.0,DN.0.0,ALL.0.0,NONESKD.0.0,DNCKD.0.0,
                                 CTRL_DNCKD.0.0,ACR.0.0,EGFR.0.0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,SEX.0.0,
                                 IDEAL_DIET2.0.0,LIFESCORE,AGE.0.0, SES_TDI.0.0,BMI.0.0,EDUYEARS,SBP.0.0,HYP_POS1,STATIN,
                                 WHR.0.0,GRS_WT_LIRd2,GRS_WT_LIRt,GRS_WT_LIRq,GRS_WT_LIRf3,GRS_WT_IRd2,
                                 GRS_WT_IRt,GRS_WT_IRq,GRS_WT_IRf3,GRS_WT_IR53d2,GRS_WT_IR53t,
                                 GRS_WT_IR53q,GRS_WT_IR53f3,GRS_WT_T2DIRd2,GRS_WT_T2DIRt,
                                 GRS_WT_T2DIRq,GRS_WT_T2DIRf3,GRS_WT_L5E8IRd2,GRS_WT_L5E8IRt,
                                 GRS_WT_L5E8IRq,GRS_WT_L5E8IRf3,GRS_WT_L1E5IRd2,GRS_WT_L1E5IRt,
                                 GRS_WT_L1E5IRq,GRS_WT_L1E5IRf3)
UKBB_AG2=as.data.frame(UKBB_AG2_m)
dim(UKBB_AG2)
rm(UKBB_AG2_m)

[1] 373487     57

In [7]:
#Dichotomize Outcomes for Logistic Regression

#1_CKD
UKBB_AG2$CKD_only.0.0 <- factor(ifelse(UKBB_AG2$CKD.0.0=="CKD controls","CKD controls",
                              ifelse(UKBB_AG2$CKD.0.0=="CKD","CKD",NA)),
                levels = c("CKD controls", "CKD"))
#Set the refernece
UKBB_AG2$CKD_only.0.0 <- relevel(UKBB_AG2$CKD_only.0.0, ref = "CKD controls")

#2_CKD Extreme
UKBB_AG2$CKD_ex.0.0 <- factor(ifelse(UKBB_AG2$CKD.0.0=="CKD controls","CKD controls",
                              ifelse(UKBB_AG2$CKD.0.0=="CKD extreme","CKD extreme",NA)),
                levels = c("CKD controls", "CKD extreme"))
#Set the refernece
UKBB_AG2$CKD_ex.0.0 <- relevel(UKBB_AG2$CKD_ex.0.0, ref = "CKD controls")

#3_Micro
UKBB_AG2$micro.0.0 <- factor(ifelse(UKBB_AG2$ALBUMINERIA.0.0=="micro","micro",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="normo","normo",NA)),
                levels = c("normo", "micro"))
#Set the reference
UKBB_AG2$micro.0.0 <- relevel(UKBB_AG2$micro.0.0, ref = "normo")

#4_Macro
UKBB_AG2$macro.0.0 <- factor(ifelse(UKBB_AG2$ALBUMINERIA.0.0=="macro","macro",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="normo","normo",NA)),
                levels = c("normo", "macro"))
#Set the reference
UKBB_AG2$macro.0.0 <- relevel(UKBB_AG2$macro.0.0, ref = "normo")

#5_Macro
UKBB_AG2$macro.0.0 <- factor(ifelse(UKBB_AG2$ALBUMINERIA.0.0=="macro","macro",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="normo","normo",NA)),
                levels = c("normo", "macro"))

#6_ESKD vs. Macro
UKBB_AG2$ESKD_macro.0.0 <- factor(ifelse(UKBB_AG2$ESKD.0.0=="yes","ESKD",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="macro","macro",NA)),
                levels = c("macro","ESKD"))

#7_DNCKD vs. Control DNCKD
UKBB_AG2$DNCKD2.0.0 <- factor(ifelse(UKBB_AG2$DNCKD.0.0=="yes","DNCKD",
                              ifelse(UKBB_AG2$CTRL_DNCKD.0.0=="yes","DNCKD Control",NA)),
                levels = c("DNCKD Control","DNCKD"))

#8_ESKD vs. Normo, Macro, Micro
UKBB_AG2$ESKD_Albu.0.0 <- factor(ifelse(UKBB_AG2$ESKD.0.0=="yes","ESKD",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0 %in% c("normo","macro","micro"),"albu",NA)),
                levels = c("albu","ESKD"))

#Set the reference
UKBB_AG2$macro.0.0 <- relevel(UKBB_AG2$macro.0.0, ref = "normo")

#Summarize Counts of Disease Outcomes
table(UKBB_AG2$CKD_only.0.0) #1
table(UKBB_AG2$CKD_ex.0.0) #2
table(UKBB_AG2$micro.0.0) #3
table(UKBB_AG2$macro.0.0) #4
table(UKBB_AG2$ESKD.0.0) #5
table(UKBB_AG2$DN.0.0) #6
table(UKBB_AG2$ALL.0.0) #7
table(UKBB_AG2$ESKD.0.0) #8
table(UKBB_AG2$ESKD_macro.0.0) #8
table(UKBB_AG2$ESKD_Albu.0.0) #9
table(UKBB_AG2$DNCKD2.0.0) #10


CKD controls          CKD 
      349669         6108 


CKD controls  CKD extreme 
      349669          984 


 normo  micro 
348496  14070 


 normo  macro 
348496   1120 


    no    yes 
356332    447 

...


    no    yes 
332345  15439 


    no    yes 
356332    447 


macro  ESKD 
  963   447 


  albu   ESKD 
346560    447 


DNCKD Control         DNCKD 
       326513           645 

In [9]:
# fmla <- as.formula(paste0('relevel(as.factor(micro.0.0),"normo")'," ~ ", 'relevel(as.factor(GRS_WT_T2DIRf3),"low risk")',"+ AGE.0.0 + SEX.0.0 + PC1 +
#                                         PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 +
#                                         + EDUYEARS + SES_TDI.0.0 + BMI.0.0 + HYP_POS1 + STATIN + T2D_status"), env = environment())
# M1 <- glm(fmla, data=UKBB_AG2, binomial(link="logit"))   
# summary(M1)
# anova(M1,test="LRT")


Call:
glm(formula = fmla, family = binomial(link = "logit"), data = UKBB_AG2)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-0.7290  -0.3079  -0.2671  -0.1902   2.9832  

Coefficients:
                                                           Estimate Std. Error
(Intercept)                                               -4.245662   0.106192
relevel(as.factor(GRS_WT_T2DIRf3), "low risk")high risk   -0.005891   0.032181
relevel(as.factor(GRS_WT_T2DIRf3), "low risk")medium risk -0.020263   0.030936
AGE.0.0                                                    0.014376   0.001271
SEX.0.0Male                                               -0.066100   0.017932
PC1                                                        0.741818   0.514428
PC2                                                       -0.149767   0.624342
PC3                                                       -1.015210   0.891549
PC4                                                        3.099866   0.991558
PC

,Df,Deviance,Resid. Df,Resid. Dev,Pr(>Chi)
,<int>,<dbl>,<int>,<dbl>,<dbl>
NULL,NA,NA,358042,116926.3,NA
"relevel(as.factor(GRS_WT_T2DIRf3), ""low risk"")",2,2.755017e+01,358040,116898.8,1.041254e-06
AGE.0.0,1,1.031530e+03,358039,115867.2,2.517485e-226
SEX.0.0,1,4.659973e+01,358038,115820.6,8.707225e-12
PC1,1,1.580955e-01,358037,115820.5,6.909161e-01
PC2,1,1.635581e+00,358036,115818.8,2.009329e-01
PC3,1,2.957798e+00,358035,115815.9,8.546424e-02
PC4,1,7.383408e+00,358034,115808.5,6.582832e-03
PC5,1,5.457902e-01,358033,115808.0,4.600430e-01


In [18]:
##MODEL APPROACH #1 - Three Risk Groups

#Previously evaluated non-linearity in prior section#
#Current section evalautes the continuous form:
RESULTS_CONT <- data.frame()
RESULTS_OR <- data.frame()

for (ii in c('relevel(as.factor(GRS_WT_T2DIRf3),"low risk")')) {
    for(kk in c('Model6')){
                   for (jj in c('relevel(as.factor(CKD_only.0.0),"CKD controls")',
                       'relevel(as.factor(CKD_ex.0.0),"CKD controls")',
                        'relevel(as.factor(micro.0.0),"normo")','relevel(as.factor(macro.0.0),"normo")',
                        'relevel(as.factor(DN.0.0),"no")','relevel(as.factor(ALL.0.0),"no")',
                        'relevel(as.factor(DNCKD2.0.0),"DNCKD Control")'
                      )) {
              
        #Used across formulas
        term <- ii   
              
        #Model 1 - Age, gender, PCI
        if(kk == "Model1"){
            
        fmla <- as.formula(paste0(jj," ~ ",term, "+ AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10"), env = environment()) }
        if(kk == "Model2"){
            
        fmla <- as.formula(paste0(jj," ~ ", term,"+ AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 +
                                        + EDUYEARS + SES_TDI.0.0"), env = environment()) }
          
        if(kk == "Model3"){
            
        fmla <- as.formula(paste0(jj," ~ ", term,"+ AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 +
                                        + EDUYEARS + SES_TDI.0.0 + BMI.0.0"), env = environment()) }
              
        if(kk == "Model4"){
            
        fmla <- as.formula(paste0(jj," ~ ", term,"+ AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 +
                                        + EDUYEARS + SES_TDI.0.0 + BMI.0.0 + HYP_POS1"), env = environment()) }
              
        if(kk == "Model5"){
            
        fmla <- as.formula(paste0(jj," ~ ", term,"+ AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 +
                                        + EDUYEARS + SES_TDI.0.0 + BMI.0.0 + STATIN"), env = environment()) }
        #Removed T2D status on 4/13/22      
        if(kk == "Model6"){
            
        fmla <- as.formula(paste0(jj," ~ ", term,"+ AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 +
                                        + EDUYEARS + SES_TDI.0.0 + BMI.0.0 + HYP_POS1 + STATIN"), env = environment()) }
          
        M1 <- glm(fmla, data=UKBB_AG2, binomial(link="logit"))                                                                                
        M1_2 <-  M1 %>% summary()
        M1_3 <- anova(M1,test="LRT")
        
        #Model 1 - Sub-groups
        #Type 2 Diabetics
        M1_T2D <- UKBB_AG2 %>% filter(T2D_status == 1) %>% glm(formula=fmla,family=binomial(link="logit"))                                                                                
        M1_T2D2 <- M1_T2D  %>% summary()
        M1_T2D3 <- anova(M1_T2D ,test="LRT")
            
        #Non-Diabetes
        M1_ND <- UKBB_AG2 %>% filter(T2D_status == 0) %>% glm(formula=fmla, family=binomial(link="logit"))                                                                                
        M1_ND2 <- M1_ND  %>% summary()
        M1_ND3 <- anova(M1_ND ,test="LRT")
        
        TEMP<- list(model=kk, var=jj,var2=ii, total=M1_2,t2d=M1_T2D2,nd=M1_ND2,lrt_tot=M1_3,
                    lrt_t2d=M1_T2D3,lrt_nd=M1_ND3)
                     
        #Confidence Intervals
        #Entire Sample
        M1$dwn_conf <- exp(coefficients(TEMP$total)[,1]-1.96*coefficients(TEMP$total)[,2])
        M1$up_conf <- exp(coefficients(TEMP$total)[,1]+1.96*coefficients(TEMP$total)[,2])
        TABLE2 <- round(cbind(OR=exp(coefficients(TEMP$total)[,1]),CI=cbind(M1$dwn_conf,M1$up_conf)),3)   
        #Diabetics
        M1_T2D$dwn_conf <- exp(coefficients(TEMP$t2d)[,1]-1.96*coefficients(TEMP$t2d)[,2])
        M1_T2D$up_conf <- exp(coefficients(TEMP$t2d)[,1]+1.96*coefficients(TEMP$t2d)[,2])
        TABLE2_T2D <- round(cbind(OR=exp(coefficients(TEMP$t2d)[,1]),CI=cbind(M1_T2D$dwn_conf,M1_T2D$up_conf)),3)
        #Non-Diabetics
        M1_ND2$dwn_conf <- exp(coefficients(TEMP$nd)[,1]-1.96*coefficients(TEMP$nd)[,2])
        M1_ND2$up_conf <- exp(coefficients(TEMP$nd)[,1]+1.96*coefficients(TEMP$nd)[,2])
        TABLE2_ND <- round(cbind(OR=exp(coefficients(TEMP$nd)[,1]),CI=cbind(M1_ND2$dwn_conf,M1_ND2$up_conf)),3)
            
        #Summary Statistics
        OR_CI <- list(model=kk, var=jj,var2=ii,nd_ci=TABLE2_ND,t2d_ci=TABLE2_T2D,all_ci=TABLE2)
        RESULTS_OR <- c(RESULTS_OR,OR_CI)    
        #Combined results
        RESULTS_CONT <- c(RESULTS_CONT,TEMP)
       }
    }
}

In [38]:
#Total Sample Score

#Intermediate and High Risk (Three Categories)
#Variable of interest listed first
#Only Save Two Coefficients

#6 = unique number of elements in list of 180 objects
#4,5,6 = model-based results (glm summary)
coef_all3 <- data.frame()
for (i in seq(from=1,to=length(RESULTS_CONT)/9,by=1)) {

  #identifiers
  model_id <- 2 + 9*(i-1) #model id
  i1 <- 4 + 9*(i-1) #total models
  i2 <- 5 + 9*(i-1) #t2d models
  i3 <- 6 + 9*(i-1) #nd models
    
  #lrt
  i7 <- 7 + 9*(i-1) #total models
  i8 <- 8 + 9*(i-1) #t2d models
  i9 <- 9 + 9*(i-1) #nd models 

    
  #model 1-6
  term <- i - (i%/%60)*60 + 1 #remainder from subtracting from multiple of 30, depend on the model?
  i4 <- ifelse(term %in% c(1:10),
               'Model 6',
               ifelse(term %in% c(11:20),'Model 6',
                      ifelse(term %in% c(21:30),'Model 6',
                            ifelse(term %in% c(31:40),'Model 6',
                                  ifelse(term %in% c(41:50),'Model 6',
                                        ifelse(term %in% c(51:60),'Model 6',))))))
  
  #ENTIRE SAMPLE 
  #pull coefficients and convert to OR
  coef_all <- data.frame(round(exp(coefficients(RESULTS_CONT[[i1]])),2)[c(2:4),1])
  id_model <- names(round(exp(coefficients(RESULTS_CONT[[i1]])),2)[c(2:4),1])
  model_id2 <- RESULTS_CONT[[model_id]]
  rep_model <- rep(model_id2,dim(coef_all)[1])
  rep_adj <- rep(i4,dim(coef_all)[1])
  
  #pull coefficients and calculate 95% CI
  up_coef <- data.frame(round(exp(coefficients(RESULTS_CONT[[i1]])[,1]+1.96*coefficients(RESULTS_CONT[[i1]])[,2]),2)[2:4])
  down_coef <- data.frame(round(exp(coefficients(RESULTS_CONT[[i1]])[,1]-1.96*coefficients(RESULTS_CONT[[i1]])[,2]),2)[2:4])
  #likelihood ratio
  lrt_total <- RESULTS_CONT[[i7]][2,5]
  lrt_total2 <- rep(as.character(lrt_total),dim(coef_all)[1])
    
  coef_all2 <- cbind(rep_model,id_model) #model outcome to model var
  coef_all2 <- cbind(coef_all2,coef_all) #coefficients
  coef_all2 <- cbind(coef_all2,rep_adj)
  coef_all2 <- cbind(coef_all2,down_coef) #95 CI
  coef_all2 <- cbind(coef_all2,up_coef) #95 CI
  coef_all2 <- cbind(coef_all2,lrt_total2)
  coef_all3 <- rbind(coef_all3,coef_all2)
}

#reformat table
#rename
names(coef_all3) <- c("rep_model","id_model","OR","model_adj","lower","upper","lrt")
#combine HR and 95% CI
coef_all3$combo <- paste0(coef_all3$OR," (",coef_all3$lower,"-",coef_all3$upper,")")
coef_all3 <- subset(coef_all3, select = -c(3,5,6))
#high/low
coef_all3$category <- coef_all3$id_model
#substring
coef_all3$sub <- substr(coef_all3$id_model,19,32)
coef_all3$rep_model2 <- substr(coef_all3$rep_model,19,35)
#key
coef_all3$key <- paste0(coef_all3$sub,"-",coef_all3$rep_model2)
#spread
coef_all3 <- subset(coef_all3, select = -c(1,2))
coef_all4 <- spread(coef_all3,key=category,value=combo)

#T2D Sample Score

#Total Sample Score

#6 = unique number of elements in list of 180 objects
#4,5,6 = model-based results (glm summary)
coef_all3 <- data.frame()
for (i in seq(from=1,to=length(RESULTS_CONT)/9,by=1)) {

  #identifiers
  model_id <- 2 + 9*(i-1) #model id
  i1 <- 4 + 9*(i-1) #total models
  i2 <- 5 + 9*(i-1) #t2d models
  i3 <- 6 + 9*(i-1) #nd models
    
  #lrt
  i7 <- 7 + 9*(i-1) #total models
  i8 <- 8 + 9*(i-1) #t2d models
  i9 <- 9 + 9*(i-1) #nd models 
    
  #model 1-6
  term <- i - (i%/%60)*60 + 1 #remainder from subtracting from multiple of 30, depend on the model?
  i4 <- ifelse(term %in% c(1:10),
               'Model 6',
               ifelse(term %in% c(11:20),'Model 6',
                      ifelse(term %in% c(21:30),'Model 6',
                            ifelse(term %in% c(31:40),'Model 6',
                                  ifelse(term %in% c(41:50),'Model 6',
                                        ifelse(term %in% c(51:60),'Model 6',))))))
  
  #ENTIRE SAMPLE 
  #pull coefficients and convert to OR
  coef_all <- data.frame(round(exp(coefficients(RESULTS_CONT[[i2]])),2)[c(2:4),1])
  id_model <- names(round(exp(coefficients(RESULTS_CONT[[i2]])),2)[c(2:4),1])
  model_id2 <- RESULTS_CONT[[model_id]]
  rep_model <- rep(model_id2,dim(coef_all)[1])
  rep_adj <- rep(i4,dim(coef_all)[1])
  
  #pull coefficients and calculate 95% CI
  up_coef <- data.frame(round(exp(coefficients(RESULTS_CONT[[i2]])[,1]+1.96*coefficients(RESULTS_CONT[[i2]])[,2]),2)[2:4])
  down_coef <- data.frame(round(exp(coefficients(RESULTS_CONT[[i2]])[,1]-1.96*coefficients(RESULTS_CONT[[i2]])[,2]),2)[2:4])
  #likelihood ratio
  lrt_total <- RESULTS_CONT[[i8]][2,5]
  lrt_total2 <- rep(as.character(lrt_total),dim(coef_all)[1])
    
  coef_all2 <- cbind(rep_model,id_model) #model outcome to model var
  coef_all2 <- cbind(coef_all2,coef_all) #coefficients
  coef_all2 <- cbind(coef_all2,rep_adj)
  coef_all2 <- cbind(coef_all2,down_coef) #95 CI
  coef_all2 <- cbind(coef_all2,up_coef) #95 CI
  coef_all2 <- cbind(coef_all2,lrt_total2)
  coef_all3 <- rbind(coef_all3,coef_all2)
}

#reformat table
#rename
names(coef_all3) <- c("rep_model","id_model","OR","model_adj","lower","upper","lrt")
#combine HR and 95% CI
coef_all3$combo <- paste0(coef_all3$OR," (",coef_all3$lower,"-",coef_all3$upper,")")
coef_all3 <- subset(coef_all3, select = -c(3,5,6))
#high/low
coef_all3$category <- coef_all3$id_model
#substring
coef_all3$sub <- substr(coef_all3$id_model,19,32)
coef_all3$rep_model2 <- substr(coef_all3$rep_model,19,35)
#key
coef_all3$key <- paste0(coef_all3$sub,"-",coef_all3$rep_model2)
#spread
coef_all3 <- subset(coef_all3, select = -c(1,2))
coef_all4_t2d <- spread(coef_all3,key=category,value=combo)

#Non-Diabetic Sample Score

#Total Sample Score

#6 = unique number of elements in list of 180 objects
#4,5,6 = model-based results (glm summary)
coef_all3 <- data.frame()
for (i in seq(from=1,to=length(RESULTS_CONT)/9,by=1)) {

  #identifiers
  model_id <- 2 + 9*(i-1) #model id
  i1 <- 4 + 9*(i-1) #total models
  i2 <- 5 + 9*(i-1) #t2d models
  i3 <- 6 + 9*(i-1) #nd models
    
  #lrt
  i7 <- 7 + 9*(i-1) #total models
  i8 <- 8 + 9*(i-1) #t2d models
  i9 <- 9 + 9*(i-1) #nd models 

    
  #model 1,2,3
  term <- i - (i%/%60)*60 + 1 #remainder from subtracting from multiple of 30, depend on the model 
  i4 <- ifelse(term %in% c(1:10),
               'Model 6',
               ifelse(term %in% c(11:20),'Model 6',
                      ifelse(term %in% c(21:30),'Model 6',
                            ifelse(term %in% c(31:40),'Model 6',
                                  ifelse(term %in% c(41:50),'Model 6',
                                        ifelse(term %in% c(51:60),'Model 6',))))))
  
  #ENTIRE SAMPLE 
  #pull coefficients and convert to OR
  coef_all <- data.frame(round(exp(coefficients(RESULTS_CONT[[i3]])),2)[c(2:4),1])
  id_model <- names(round(exp(coefficients(RESULTS_CONT[[i3]])),2)[c(2:4),1])
  model_id2 <- RESULTS_CONT[[model_id]]
  rep_model <- rep(model_id2,dim(coef_all)[1])
  rep_adj <- rep(i4,dim(coef_all)[1])
  
  #pull coefficients and calculate 95% CI
  up_coef <- data.frame(round(exp(coefficients(RESULTS_CONT[[i3]])[,1]+1.96*coefficients(RESULTS_CONT[[i3]])[,2]),2)[2:4])
  down_coef <- data.frame(round(exp(coefficients(RESULTS_CONT[[i3]])[,1]-1.96*coefficients(RESULTS_CONT[[i3]])[,2]),2)[2:4])
  #likelihood ratio
  lrt_total <- RESULTS_CONT[[i9]][2,5]
  lrt_total2 <- rep(as.character(lrt_total),dim(coef_all)[1])
    
  coef_all2 <- cbind(rep_model,id_model) #model outcome to model var
  coef_all2 <- cbind(coef_all2,coef_all) #coefficients
  coef_all2 <- cbind(coef_all2,rep_adj)
  coef_all2 <- cbind(coef_all2,down_coef) #95 CI
  coef_all2 <- cbind(coef_all2,up_coef) #95 CI
  coef_all2 <- cbind(coef_all2,lrt_total2)
  coef_all3 <- rbind(coef_all3,coef_all2)
}

#reformat table
#rename
names(coef_all3) <- c("rep_model","id_model","OR","model_adj","lower","upper","lrt")
#combine HR and 95% CI
coef_all3$combo <- paste0(coef_all3$OR," (",coef_all3$lower,"-",coef_all3$upper,")")
coef_all3 <- subset(coef_all3, select = -c(3,5,6))
#high/low
coef_all3$category <- coef_all3$id_model
#substring
coef_all3$sub <- substr(coef_all3$id_model,19,32)
coef_all3$rep_model2 <- substr(coef_all3$rep_model,19,35)
#key
coef_all3$key <- paste0(coef_all3$sub,"-",coef_all3$rep_model2)
#spread
coef_all3 <- subset(coef_all3, select = -c(1,2))
coef_all4_nd <- spread(coef_all3,key=category,value=combo)

#Export the Sheet
write.csv(coef_all4,'/salemlab/users/agarduno/jupyter/Analysis/All_MVMR_12April22_T2DP.txt')
write.csv(coef_all4_t2d,'/salemlab/users/agarduno/jupyter/Analysis/T2D_MVMR_12April22_T2DP.txt')
write.csv(coef_all4_nd,'/salemlab/users/agarduno/jupyter/Analysis/ND_MVMR_12April22_T2DP.txt')